In [68]:
import pandas as pd
import kagglehub
import re

In [69]:
blastchar_path = kagglehub.dataset_download("blastchar/telco-customer-churn")
print("Path dataset:", blastchar_path)

Path dataset: /home/heycaio/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1


### Telco customer churn

In [70]:
customer_df = pd.read_csv('/home/heycaio/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [71]:
customer_df.shape

(7043, 21)

In [72]:
customer_df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [73]:
customer_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Remover colunas que não entram no modelo

In [74]:
df = customer_df.copy()

cols_to_drop = [
    "customerID",
    "PhoneService",
    "MultipleLines",
    "InternetService",
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

### Padronizar nomes de colunas

In [75]:
def camel_to_snake(name: str) -> str:
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    
    return s2.lower()

df.columns = [camel_to_snake(c) for c in df.columns]

In [76]:
df.head(2)

,gender,senior_citizen,partner,dependents,tenure,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


### Campo churn Yes/No para 1/0

In [77]:
df["churn"] = df["churn"].map({"Yes": 1, "No": 0}).astype(int)

### Converter TotalCharges para numérico

In [78]:
df["total_charges"] = pd.to_numeric(df["total_charges"], errors="coerce")

# Clientes com tenure = 0 normalmente têm TotalCharges vazio
df["total_charges"] = df["total_charges"].fillna(0.0)

### Remover colunas duplicatas

In [79]:
df = df.drop_duplicates()

### Tratar nulos numéricos

In [80]:
numeric_cols = ["tenure", "monthly_charges", "total_charges"]

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

In [81]:
print("Shape final:", df.shape)

print("\nDistribuição do churn:")
print(df["churn"].value_counts(normalize=True))

print("\nValores nulos restantes:")
print(df.isna().sum().sort_values(ascending=False).head(10))

Shape final: (7021, 17)

Distribuição do churn:
churn
0    0.735508
1    0.264492
Name: proportion, dtype: float64

Valores nulos restantes:
gender               0
senior_citizen       0
partner              0
dependents           0
tenure               0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
dtype: int64


In [82]:
df.to_csv("churn_clean_datasete.csv", index=False)


In [84]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# =========================
# 1) Separar X e y
# =========================
X = df.drop(columns=["churn"])
y = df["churn"]

# =========================
# 2) Identificar tipos de variáveis
# =========================
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "string"]).columns.tolist()

# =========================
# 3) Pipelines por tipo
# =========================
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# =========================
# 4) ColumnTransformer (pipeline final)
# =========================
preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)

# =========================
# 5) Teste rápido do pipeline
# =========================
X_processed = preprocessing_pipeline.fit_transform(X)

print("Shape antes:", X.shape)
print("Shape depois do pipeline:", X_processed.shape)


Shape antes: (7021, 16)
Shape depois do pipeline: (7021, 37)
